In [1]:
import pandas as pd
import pandahouse as ph

In [2]:
connection_default = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

### Задача 1

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [3]:
# Посмотрим таблицу

In [4]:
query_1 = """ 
SELECT
    st_id,
    toStartOfDay(timest) AS day,
    correct,
    subject
FROM default.peas
"""

In [5]:
df = ph.read_clickhouse(query = query_1, connection = connection_default)
df.head()

,st_id,day,correct,subject
0,100379,2021-10-30,1,Theory of probability
1,100379,2021-10-30,0,Vizualization
2,100379,2021-10-30,1,Theory of probability
3,100379,2021-10-30,1,Vizualization
4,100379,2021-10-30,1,Theory of probability


In [6]:
# какой временной интервал в?

In [7]:
query_2 = """
SELECT DISTINCT toStartOfDay(timest) AS time
FROM default.peas 
"""

In [8]:
time = ph.read_clickhouse(query = query_2, connection = connection_default)
time.head()

,time
0,2021-10-30
1,2021-10-31


In [9]:
# информация за дня - значит берем всю информацию

In [10]:
query_3 = """
SELECT COUNT(count_gor) AS count_20
FROM ( 
SELECT 
    st_id,
    COUNT( correct) AS count_gor
FROM default.peas
WHERE correct = 1
GROUP BY st_id
HAVING count_gor > 19
    )
"""

In [11]:
count_20 = ph.read_clickhouse(query = query_3, connection = connection_default)
count_20

,count_20
0,136


### Задача 2

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Даны таблицы: default.peas (см. выше), default.studs и default.final_project_check

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

1. ARPU 
2. ARPAU 
3. CR в покупку 
4. СR активного пользователя в покупку 
5. CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [12]:
# Посмотрим таблицу default_studs

In [13]:
query_2_1 = """ 
SELECT *
FROM default.studs
"""

In [14]:
default_studs = ph.read_clickhouse(query = query_2_1, connection = connection_default)
default_studs.head()

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot
3,104885,pilot
4,104966,pilot


In [15]:
query_2_2 = """ 
SELECT
    st_id,
    toStartOfDay(sale_time) AS day,
    money,
    subject
FROM default.final_project_check
"""

In [16]:
default_final = ph.read_clickhouse(query = query_2_2, connection = connection_default)
default_final.head()

,st_id,day,money,subject
0,101432,2021-10-31,85000,Math
1,101432,2021-10-31,65000,Vizualization
2,104885,2021-10-30,65000,Vizualization
3,104885,2021-10-30,75000,Statistics
4,106464,2021-10-31,85000,Math


In [17]:
#arpu доход со всех пользователей/ на количество всех пользователей

In [18]:
# ARPAU (Average Revenue Per Active User) - средний платеж на всех активных пользователей. 
# количесто активных пользователей/ на сумму покупок активных пользователей

In [19]:
# CR в покупку - количетсво купивших/ на общее количество

In [20]:
# CR активного пользователя в покупку - количество купивших с 10+ задач/ на общее количество c 10+ задач

In [21]:
# CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике -
# количество решивших 2+ задача по математике и кувивших math / на количество решивши 2+ задач

In [22]:
query_2_3 = """
SELECT 
    s.test_grp,
    SUM(money)/uniqExact(s.st_id)  AS arpu,
    sumIf(money, sum_gor > 10)/uniqIf(s.st_id, p.sum_gor > 10) AS arpau,
    100*uniqIf(s.st_id, money > 0)/uniqExact(s.st_id) AS CR_purch,
    100*uniqIf(s.st_id, money > 0 AND sum_gor > 10)/uniqIf(s.st_id, sum_gor > 10) AS CR_purch_act,
    100*uniqIf(s.st_id, subject = 'Math' AND sum_math >= 2)/uniqIf(s.st_id,sum_math >= 2) AS CR_math
FROM
    default.studs AS s
    LEFT JOIN
    default.final_project_check AS f
    ON s.st_id = f.st_id
    LEFT JOIN
    (
    SELECT 
        st_id,
        SUM(correct) AS sum_gor,
        sumIf(correct, subject = 'Math') AS sum_math
    FROM default.peas
    GROUP BY st_id
    ) AS p
    ON s.st_id = p.st_id
GROUP BY test_grp
    
"""
metric = ph.read_clickhouse(query = query_2_3, connection = connection_default)
metric.head()

,s.test_grp,arpu,arpau,CR_purch,CR_purch_act,CR_math
0,control,4540.983607,10393.700787,4.918033,11.023622,6.122449
1,pilot,11508.474576,29739.583333,10.847458,26.041667,9.523810


In [23]:
# попробуем сделать без комбинаторов агрегатных функций из clickhouse

Какие данные нам нужны:
1. Сумма покупок
2. Количество пользователей
3. Сумма покупок пользователей сделавших более 10 уроков
4. Количество пользователей сделавших более 10 уроков
5. Количество пользователей сделавших покупку
6. Количество пользователей сделавших покупку и сделавших более 10 уроков
7. Количество пользователей купивших математику и сделавших 2 и более урока по математике
8. Количество пользователей сделавших 2 и более урока по математике

In [24]:
query_2_7 = """
SELECT 
    st.test_grp,
    SUM(f.money_id)/uniqExact(st.st_id) AS arpu,
    SUM(CASE WHEN sum_gor > 10 then f.money_id END)/SUM(CASE WHEN sum_gor > 10 then 1 END) AS arpau,
    SUM(CASE WHEN f.money_id > 0 then 1 end)/COUNT(DISTINCT st.st_id) AS CR_purch,
    SUM(CASE WHEN f.money_id > 0 AND sum_gor > 10 THEN 1 END)/
    SUM(CASE WHEN sum_gor > 10 THEN 1 END)*100 AS CR_purch_act,
    SUM(CASE WHEN math > 0 AND sum_math >= 2 THEN 1 END)/
    SUM(CASE WHEN sum_math >= 2 THEN 1 END)*100 AS CR_math
    
    
FROM
    default.studs AS st
    LEFT JOIN
    (
    SELECT 
        st_id,
        SUM(correct) AS sum_gor,
        SUM(CASE WHEN subject = 'Math' AND correct = 1 THEN 1 END) AS sum_math
    FROM default.peas
    GROUP BY st_id
    ) AS s_m
    ON st.st_id = s_m.st_id
    LEFT JOIN
    (
    SELECT 
        st_id, 
        SUM(money) AS money_id,
        SUM(CASE WHEN subject = 'Math' THEN 1 END) AS math
    FROM default.final_project_check
    GROUP BY st_id
    ) AS f
    ON st.st_id = f.st_id
GROUP BY st.test_grp
    """
answer = ph.read_clickhouse(query = query_2_7, connection = connection_default)
answer.head()

,st.test_grp,arpu,arpau,CR_purch,CR_purch_act,CR_math
0,control,4540.983607,10393.700787,0.049180,11.023622,6.122449
1,pilot,11508.474576,29739.583333,0.108475,26.041667,9.523810


In [25]:
# сделаем через WITH

In [26]:
query_2_8 = """
WITH 
    peas AS (
        SELECT 
            st_id,
            SUM(correct) AS sum_gor,
            SUM(CASE WHEN subject = 'Math' AND correct = 1 THEN 1 END) AS sum_math
        FROM default.peas
        GROUP BY st_id),
    
    final AS (
        SELECT 
            st_id, 
            SUM(money) AS money_id,
            SUM(CASE WHEN subject = 'Math' THEN 1 END) AS math
        FROM default.final_project_check
        GROUP BY st_id)
    

SELECT 
    st.test_grp,
    SUM(f.money_id)/uniqExact(st.st_id) AS arpu,
    SUM(CASE WHEN sum_gor > 10 then f.money_id END)/SUM(CASE WHEN sum_gor > 10 then 1 END) AS arpau,
    SUM(CASE WHEN f.money_id > 0 then 1 end)/COUNT(DISTINCT st.st_id) AS CR_purch,
    SUM(CASE WHEN f.money_id > 0 AND sum_gor > 10 THEN 1 END)/
    SUM(CASE WHEN sum_gor > 10 THEN 1 END)*100 AS CR_purch_act,
    SUM(CASE WHEN math > 0 AND sum_math >= 2 THEN 1 END)/
    SUM(CASE WHEN sum_math >= 2 THEN 1 END)*100 AS CR_math
    
    
FROM
    default.studs AS st
    LEFT JOIN
    peas AS p
    ON st.st_id = p.st_id
    LEFT JOIN
    final as f
    ON st.st_id = f.st_id
GROUP BY 
    st.test_grp
    """
answer_with = ph.read_clickhouse(query = query_2_8, connection = connection_default)
answer_with.head()

,st.test_grp,arpu,arpau,CR_purch,CR_purch_act,CR_math
0,control,4540.983607,10393.700787,0.049180,11.023622,6.122449
1,pilot,11508.474576,29739.583333,0.108475,26.041667,9.523810
